In [1]:
# import libraries
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# Set project root and change working directory
project_root = Path.cwd().parent.parent  # src/notebook/ is two levels below root
os.chdir(project_root)  # Change to project root so PriceManager can find data/
print(f"📁 Working directory: {os.getcwd()}")

# Add src directory to Python path
sys.path.insert(0, str(project_root / "src"))

from tiingo import TiingoClient
from core.config import Config
from market import PriceManager
from universe import SP500Universe

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

# Initialize configuration and components
config = Config("config/settings.yaml")
universe = SP500Universe()

# Initialize Tiingo client and PriceManager
tiingo = TiingoClient({
    'api_key': config.get('fetcher.tiingo.api_key'),
    'session': True
})

price_mgr = PriceManager(tiingo=tiingo, universe=universe)

print("✅ Components initialized successfully")

📁 Working directory: /Users/frank/Projects/QuantX/quantx-data-builder
✅ Libraries imported successfully
✅ Components initialized successfully


In [2]:

'''
Read monthly prices of META and S&P 500 index (SPY) for 5-year beta calculation
5 years of monthly data ending October 2025
start period: 2020-11-01 (5 years back)
end period: 2025-10-31 (last complete month)
period format: YYYY-MM-DD
'''
spy_monthly = price_mgr.load_price_data(
    symbol='SPY',
    frequency='monthly',
    start_date='2020-11-01',
    end_date='2025-10-31'
)

meta_monthly = price_mgr.load_price_data(
    symbol='META',
    frequency='monthly',
    start_date='2020-11-01',
    end_date='2025-10-31'
)

print(f"SPY columns: {spy_monthly.columns.tolist()}")
print(f"META columns: {meta_monthly.columns.tolist()}")

# joining the closing prices of the two datasets with same date index
# Both datasets are indexed by date, ensuring proper alignment
monthly_prices = pd.DataFrame({
    'META': meta_monthly.set_index('date')['adj_close'],
    'SPY': spy_monthly.set_index('date')['adj_close']
})

print(f"\n📊 Monthly prices shape: {monthly_prices.shape}")
print(f"Date range: {monthly_prices.index.min()} to {monthly_prices.index.max()}")
print(f"Number of matching dates: {len(monthly_prices.dropna())}")
print(monthly_prices.head())

# calculate monthly returns
monthly_returns = monthly_prices.pct_change(1)
clean_monthly_returns = monthly_returns.dropna(axis=0)  # drop first missing row
print(clean_monthly_returns.head())


SPY columns: ['gvkey', 'date', 'open', 'high', 'low', 'close', 'volume', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume', 'div_cash', 'split_factor', 'exchange', 'currency', 'freq', 'year']
META columns: ['gvkey', 'date', 'open', 'high', 'low', 'close', 'volume', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume', 'div_cash', 'split_factor', 'exchange', 'currency', 'freq', 'year']

📊 Monthly prices shape: (60, 2)
Date range: 2020-11-30 to 2025-10-31
Number of matching dates: 60
                  META         SPY
date                              
2020-11-30  275.287322  337.442343
2020-12-31  271.500469  349.949980
2021-01-29  256.760566  346.383837
2021-02-26  256.054880  356.015230
2021-03-31  292.740640  372.180116
                META       SPY
date                          
2020-12-31 -0.013756  0.037066
2021-01-29 -0.054291 -0.010190
2021-02-26 -0.002748  0.027806
2021-03-31  0.143273  0.045405
2021-04-30  0.103725  0.052910


In [3]:
# split dependent and independent variable
X = clean_monthly_returns['SPY']
y = clean_monthly_returns['META']

# Add a constant to the independent value
X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y, X1)

# fit model and print results
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   META   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     15.91
Date:                Sun, 23 Nov 2025   Prob (F-statistic):           0.000191
Time:                        13:19:09   Log-Likelihood:                 49.334
No. Observations:                  59   AIC:                            -94.67
Df Residuals:                      57   BIC:                            -90.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0055      0.014      0.382      0.7